**Step 1**

Script Python che itera in ordine alfabetico sui file della cartella files e, a seconda del tipo (audio, documento, immagine), li sposti nella relativa sottocartella (qui sotto trovi un esempio). 
Se la sottocartella non esiste, il tuo script dovrà crearla automaticamente.

Durante il ciclo, lo script deve stampare le informazioni dei file: nome, tipo e dimensione in byte.

Oltre a stamparne le informazioni via via che li sposti, tieni traccia dei file creando un documento *recap.csv* con le stesse informazioni. 

La struttura finale della cartella files dovrà essere:

        - files            
            - audio
                - song1.mp3
                - song2.mp3
            - docs
                - ciao.txt
                - pippo.odt
            - images
                - bw.png
                - daffodil.jpg
                - trump.jpeg    
            - recap.csv

In [14]:
#importo librerie
#step 1
import os 
import shutil
import csv
#step 3
from PIL import Image
from tabulate import tabulate
import numpy as np

Funzione che dato in input il nome del file, e la tipologia:
1. stampa le info 
2. aggiungo le informazioni nel file di recap 
3. muove il file nella directory giusta 

In [15]:
def processo_file(file_nome, path, tipo_file):
    #divido il file_nome ogni volta che trovo un punto utilizzando la funzione split
    nome = file_nome.split(".")
    #recupero la dimensione del file utlizzando la funzione getsize di os
    dimensione = os.path.getsize(os.path.join(path, file_nome))
    #creo una var testo che memorizza il nome senza l'estensione, il tipo e la dimensione del file
    testo = "{} tipo:{} dimensioni:{}B ".format(nome[0],tipo_file, dimensione)
    #stampo testo con le informazioni
    print(testo)
    
    #scrivo le informazioni nel file racap.csv
    scrittura = csv.writer(recap_file)
    #scrivo nel file il nome senza l'estensione, il tipo e la dimensione del file utilizzando la funzione writerow
    scrittura.writerow([nome[0], "{}".format(tipo_file), "{}".format(dimensione)])
        
    #sposto il file utilizzando la funzione move
    shutil.move(os.path.join(path, file_nome), os.path.join(path, tipo_file))

Itera su i file nella cartella "files"

In [16]:
#variabile che contiene il file della directory
directory = "files"

#creo le cartelle nel caso non esistessero 
for dirnome in ["audio", "docs", "images"]:
    if not os.path.isdir(os.path.join(directory, dirnome)): 
        #se non esiste la cartella iterata dal for creo la sottocartella di directory con la funzione mkdir
        os.mkdir(os.path.join(directory, dirnome))
        
#creo il file di recap.csv  
if not os.path.exists(os.path.join(directory, "recap.csv")):  
    #se non esiste
    recap_file=open(os.path.join(directory, "recap.csv"), "w")
    #creo il file 'w' sta per write 
    scrittura = csv.writer(recap_file)
    #scrivo nel file name, type, size B utilizzando la funzione writerow
    scrittura.writerow(["name", "type", "size(B)"])
else:
    #apro recap        
    recap_file=open(os.path.join(directory, "recap.csv"), "a")
    
#dizionario estensioni files
extension_files = {
        '.jpeg': 'images',
        '.jpg': 'images',
        '.png': 'images',
        '.txt': 'docs',
        '.odt': 'docs',
        '.mp3': 'audio'
}
        
#itero tutti i file delle directory 
for file in os.listdir(directory):
    #separo nome file da estensione 
    name, ext = os.path.splitext(file)
    if ext in extension_files:
        path_file = extension_files[ext]
        processo_file(file, directory, path_file)
    else:
        print(file +" Errore: Estensione non supportata!")

        
#chiudo file recap        
recap_file.close()

song1 tipo:audio dimensioni:1087849B 
ciao tipo:docs dimensioni:12B 
song2 tipo:audio dimensioni:764176B 
.DS_Store Errore: Estensione non supportata!
images Errore: Estensione non supportata!
daffodil tipo:images dimensioni:24657B 
docs Errore: Estensione non supportata!
audio Errore: Estensione non supportata!
recap.csv Errore: Estensione non supportata!
trump tipo:images dimensioni:10195B 
bw tipo:images dimensioni:94926B 
pippo tipo:docs dimensioni:8299B 
eclipse tipo:images dimensioni:64243B 


**Step 3**

Una immagine in scala di grigio ha un solo livello di colore, una RGB ne ha 3, una RGBA 4 (l'ultimo è detto canale alpha).

Il modulo Image della libreria PIL permette di caricare un'immagine, che può essere trasformata in un array NumPy attraverso la funzione np.array. A partire da tale array, è possibile capire se l'immagine caricata è in scala di grigio, RGB o RGBA.

Aggiungi al notebook dello Step 1 uno script che iteri sulla sottocartella images e costruisca una tabella riassuntiva come questa (prodotta con la libreria tabulate).



In [17]:
#creo la var directory che identifica la directory images
directory_img = "files/images"
#creo tabella vuota
tabella = []

#itero i file all'interno della directory
for file in os.listdir(directory_img):
    #creo e inizializzo a 0 le variabili del colore
    grayscale, R, G, B, ALPHA = 0, 0, 0, 0, 0
    #apro il file utilizzando la funzione open della libreria Image 
    img = Image.open(os.path.join(directory_img, file))
    #trasformo il file in un array NumPy attraverso la funzione np.array
    np_array_img = np.array(img)
    #ottengo le dimensioni dell'array
    np_array_img_dimensioni = np_array_img.shape
    #ottengo il numero di dimensioni
    np_array_img_num_dim = len(np_array_img_dimensioni)
    #ottengo axis array
    np_array_img_axis = np.mean(np_array_img, axis=(0, 1))
    #controllo la dimensione dell'array che sto iterando
    if np_array_img_num_dim == 3:
        if np_array_img_dimensioni[2] == 3:
            #assegno i valori corrispondenti alla scala di colori
            R, G, B = np_array_img_axis[0], np_array_img_axis[1], np_array_img_axis[2]
        elif np_array_img_dimensioni[2] == 4:
            #assegno i valori corrispondenti alla scala di colori con ALPHA
            R, G, B, ALPHA = np_array_img_axis[0], np_array_img_axis[1], np_array_img_axis[2], np_array_img_axis[3]
    elif np_array_img_num_dim == 2:
        #assegno il valore corrispondente alla scala di grigio
        grayscale = np_array_img_axis
    
    #divido il nome file ogni volta che trovo un punto utilizzando la funzione split
    nome = file.split(".")
    #appendo infondo alla tabelle i dati del file che sto iterando
    tabella.append([nome[0],np_array_img.shape[0], np_array_img.shape[1],grayscale,R,G,B,ALPHA])

#stampo la tabella
print(tabulate(tabella, 
               #header
               headers = ["name", "heights", "width", "grayscale", "R", "G", "B", "ALPHA"],
               #formato tabella
               tablefmt="fancy_grid",
               floatfmt=".2f")) 

╒══════════╤═══════════╤═════════╤═════════════╤════════╤════════╤═══════╤═════════╕
│ name     │   heights │   width │   grayscale │      R │      G │     B │   ALPHA │
╞══════════╪═══════════╪═════════╪═════════════╪════════╪════════╪═══════╪═════════╡
│ daffodil │       500 │     335 │        0.00 │ 109.25 │  85.56 │  4.97 │    0.00 │
├──────────┼───────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ trump    │       183 │     275 │        0.00 │  97.01 │  98.99 │ 90.92 │    0.00 │
├──────────┼───────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ bw       │       512 │     512 │       21.48 │   0.00 │   0.00 │  0.00 │    0.00 │
├──────────┼───────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ eclipse  │       256 │     256 │        0.00 │ 109.05 │ 109.52 │ 39.85 │  133.59 │
╘══════════╧═══════════╧═════════╧═════════════╧════════╧════════╧═══════╧═════════╛
